In [10]:
%matplotlib widget
from matplotlib import pylab as plt
from ipywidgets import VBox, HBox, Layout, FloatSlider, Button, Label, interactive_output
from IPython.display import display
from collections import deque
import time
import pandas as pd
from math import pi, atan2, asin, cos, sin # Needed for SparkAgent context
import socket, struct
from threading import Thread
from sexpr import str2sexpr # Assuming sexpr.py is available or included

# --- DEFINITIONS FROM YOUR CONTEXT (Necessary to run the SparkAgent) ---
DEG_TO_RAD = pi / 180
# Simplified SparkAgent/Perception/Action definitions must be available here or imported
# (Based on the long context you provided, these classes must be defined/imported 
# before PIDAgent can inherit from them. Since the context was long, 
# I will assume you have a file or previous cell defining SparkAgent, Perception, and Action.)

# --- ASSUMED MINIMAL AGENT SETUP (Replace with your actual imports if needed) ---
# NOTE: If SparkAgent is not defined, this will fail. You must ensure
# the SparkAgent class (and its dependencies) are runnable here.
class DummySparkAgent:
    """Placeholder for the SparkAgent class context."""
    def __init__(self, *args, **kwargs):
        self.target_joints = {} # Placeholder for target joints
        # Minimal attributes needed for initialization
        self.joint_cmd_names = {'HeadYaw': "he1", 'HeadPitch': "he2", 'LShoulderPitch': "lae1", 'LShoulderRoll': "lae2", 'LElbowYaw': "lae3", 'LElbowRoll': "lae4", 'LHipYawPitch': "lle1", 'LHipRoll': "lle2", 'LHipPitch': "lle3", 'LKneePitch': "lle4", 'LAnklePitch': "lle5", 'LAnkleRoll': "lle6", 'RShoulderPitch': "rae1", 'RShoulderRoll': "rae2", 'RElbowYaw': "rae3", 'RElbowRoll': "rae4", 'RHipYawPitch': "rle1", 'RHipRoll': "rle2", 'RHipPitch': "rle3", 'RKneePitch': "rle4", 'RAnklePitch': "rle5", 'RAnkleRoll': "rle6", 'LHand': "lhand", 'RHand': "rhand"} # Added hands
        
    def start(self):
        print("Agent Started (Placeholder)")
    def sense_think_act(self):
        pass # Placeholder for the loop
    @property
    def joint_names(self):
        return list(self.joint_cmd_names.keys())

# --- THE PID AGENT (Assumes it inherits from SparkAgent, using JOINT_CMD_NAMES) ---
class PIDAgent(DummySparkAgent): # Replace DummySparkAgent with your actual SparkAgent class
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        # Initialize target_joints with 0.0 for all joints to prevent KeyError
        self.target_joints = {name: 0.0 for name in self.joint_cmd_names.keys()}
        
# --- END AGENT SETUP ---

# --- JOINT NAMES (Use the comprehensive list) ---
JOINT_CMD_NAMES = {'HeadYaw': "he1", 'HeadPitch': "he2", 'LShoulderPitch': "lae1", 'LShoulderRoll': "lae2", 'LElbowYaw': "lae3", 'LElbowRoll': "lae4", 'LHipYawPitch': "lle1", 'LHipRoll': "lle2", 'LHipPitch': "lle3", 'LKneePitch': "lle4", 'LAnklePitch': "lle5", 'LAnkleRoll': "lle6", 'RShoulderPitch': "rae1", 'RShoulderRoll': "rae2", 'RElbowYaw': "rae3", 'RElbowRoll': "rae4", 'RHipYawPitch': "rle1", 'RHipRoll': "rle2", 'RHipPitch': "rle3", 'RKneePitch': "rle4", 'RAnklePitch': "rle5", 'RAnkleRoll': "rle6", 'LHand': "lhand", 'RHand': "rhand"}

agent = PIDAgent() 
JOINT_NAMES = list(JOINT_CMD_NAMES.keys())

In [11]:
## Global State and Slider Creation (Cell 2)

# Dictionary to store recorded keyframes: {time: {JointName: Angle, ...}}
MOTION_DATA = {}
START_TIME = time.time()
IS_RECORDING = False

JOINT_SLIDERS = {} 

def update_agent_targets(**kwargs):
    """Updates the agent's internal state every time a slider is moved."""
    global agent
    # kwargs will contain {joint_name: value} for the slider that moved
    for name, value in kwargs.items():
        if name in agent.target_joints:
            agent.target_joints[name] = value

# Create the interactive function interface (links sliders to update_agent_targets)
for name in JOINT_NAMES:
    is_hand = 'Hand' in name
    
    slider = FloatSlider(
        # Standard range is -2.0 to 2.0 radians, hands 0.0 (open) to 1.0 (closed)
        min=0.0 if is_hand else -2.0, 
        max=1.0 if is_hand else 2.0, 
        step=0.01, 
        value=0.0, 
        description=name, 
        layout=Layout(width='95%')
    )
    JOINT_SLIDERS[name] = slider
    
    # Link the slider to the update function
    # NOTE: The widget name (key) must match the keyword argument in update_agent_targets
    interactive_output(update_agent_targets, {name: slider})

In [12]:
## Keyframe Recorder UI Display (Cell 3)

# --- Grouping Joints ---
head_joints = [name for name in JOINT_NAMES if name.startswith('Head')]
larm_joints = [name for name in JOINT_NAMES if name.startswith('L') and ('Elbow' in name or 'Shoulder' in name or 'Wrist' in name)]
rarm_joints = [name for name in JOINT_NAMES if name.startswith('R') and ('Elbow' in name or 'Shoulder' in name or 'Wrist' in name)]
hand_joints = [name for name in JOINT_NAMES if 'Hand' in name]
lleg_joints = [name for name in JOINT_NAMES if name.startswith('L') and ('Hip' in name or 'Knee' in name or 'Ankle' in name)]
rleg_joints = [name for name in JOINT_NAMES if name.startswith('R') and ('Hip' in name or 'Knee' in name or 'Ankle' in name)]


# --- VBox Containers ---
head_controls = VBox([Label("Head Joints")] + [JOINT_SLIDERS[name] for name in head_joints])
larm_controls = VBox([Label("Left Arm Joints")] + [JOINT_SLIDERS[name] for name in larm_joints])
rarm_controls = VBox([Label("Right Arm Joints")] + [JOINT_SLIDERS[name] for name in rarm_joints])
hand_controls = VBox([Label("Hand Joints")] + [JOINT_SLIDERS[name] for name in hand_joints])
lleg_controls = VBox([Label("Left Leg Joints")] + [JOINT_SLIDERS[name] for name in lleg_joints])
rleg_controls = VBox([Label("Right Leg Joints")] + [JOINT_SLIDERS[name] for name in rleg_joints])


# --- Control Buttons and Functions ---
record_button = Button(description="Record Keyframe (R)", button_style='success')
reset_button = Button(description="Reset Motion Data", button_style='danger')
status_label = Label(value="Motion Data: 0 Keyframes")

def record_keyframe(b):
    global MOTION_DATA
    current_time = time.time() - START_TIME
    keyframe = {name: slider.value for name, slider in JOINT_SLIDERS.items()}
    MOTION_DATA[current_time] = keyframe
    status_label.value = f"Motion Data: {len(MOTION_DATA)} Keyframes (Last: {current_time:.2f}s)"

def reset_data(b):
    global MOTION_DATA, START_TIME
    MOTION_DATA = {}
    START_TIME = time.time()
    status_label.value = "Motion Data: 0 Keyframes (RESET)"

record_button.on_click(record_keyframe)
reset_button.on_click(reset_data)


# --- Final UI Layout ---
control_panel = VBox([
    HBox([record_button, reset_button]),
    status_label,
    # Arms and Head in one row
    HBox([head_controls, larm_controls, rarm_controls, hand_controls], layout=Layout(justify_content='space-around')),
    # Legs in a second row
    HBox([lleg_controls, rleg_controls], layout=Layout(justify_content='space-around'))
])

display(control_panel)

In [13]:
## Agent Loop Start and Export Function (Cell 4)

# Start the agent (runs in a separate thread)
agent.start()
print("Agent thread started. Control the robot using the sliders above.")

def export_keyframes_to_python(function_name="recorded_motion"):
    """
    Exports the recorded MOTION_DATA into a Python function string 
    (Simplified keyframe export).
    """
    if not MOTION_DATA:
        return "No keyframes recorded to export."

    # Convert dictionary to DataFrame for easy processing
    df = pd.DataFrame.from_dict(MOTION_DATA, orient='index')
    df.index.name = 'Time'
    df = df.sort_index()
    
    output = f"def {function_name}():\n"
    output += "    names = list()\n"
    output += "    times = list()\n"
    output += "    keys = list()\n\n"
    
    # Get unique joint names in sorted order
    sorted_joint_names = sorted(df.columns)
    
    # Times (timestamps)
    times = df.index.tolist()
    
    for name in sorted_joint_names:
        output += f'    names.append("{name}")\n'
        
        # Get the corresponding joint angles for all recorded times
        angles = df[name].tolist()
        
        # The times list is the same for all joints
        output += f'    times.append({[round(t, 4) for t in times]})\n'
        
        # Keys (Angles) - simplified list of angles
        output += f'    keys.append({[round(a, 4) for a in angles]})\n\n'

    output += "    return names, times, keys\n"
    
    return output

# Example usage:
# python_code = export_keyframes_to_python(function_name="my_waving_motion")
# print(python_code)

Agent Started (Placeholder)
Agent thread started. Control the robot using the sliders above.
